### Sample notebook for CNN learning  

#### Import libraries  

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Dropout,Flatten,Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.utils import to_categorical

#### Parameters   

In [ ]:
w=56
work_dir='data/dogs_cats'
npy_data_base='dogs_cats-data'
npy_labels_base='dogs_cats-labels'
classes=['dogs','cats']
num_classes=len(classes)

model_file="dogs_cats-model-w{0}.h5".format(w)
history_file="dogs_cats-history-w{0}.csv".format(w)

#### Read train/test data from npy file
X: 0-1 (float16), y: one-hot encoding  

In [ ]:
mode="train"
npy_data_file="{0}/{1}-{2}-w{3}.npy".format(work_dir,npy_data_base,mode,w)
npy_labels_file="{0}/{1}-{2}-w{3}.npy".format(work_dir,npy_labels_base,mode,w)
X_train=np.load(npy_data_file).astype("float16")
X_train/=255
y_train=np.load(npy_labels_file)
y_train=to_categorical(y_train,num_classes)

mode="test"
npy_data_file="{0}/{1}-{2}-w{3}.npy".format(work_dir,npy_data_base,mode,w)
npy_labels_file="{0}/{1}-{2}-w{3}.npy".format(work_dir,npy_labels_base,mode,w)
X_test=np.load(npy_data_file).astype("float16")
X_test/=255
y_test=np.load(npy_labels_file)
y_test=to_categorical(y_test,num_classes)

print( X_train.shape )
print( y_train.shape )
print( X_test.shape )
print( y_test.shape )
img_rows=X_train.shape[1]
img_cols=X_train.shape[2]
img_channels=X_train.shape[3]
print("image_size:", img_rows, img_cols)
print("image_channels:", img_channels)

#### Build CNN structure  

In [ ]:
input_shape=(img_rows,img_cols,img_channels)
model=Sequential()
model.add(Conv2D(16, (3,3), activation='relu', padding='same',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

#### Training of CNN  

In [ ]:
%%time
n_epochs=40
val_split=0.2
batch_size=128
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])
cl=CSVLogger(history_file)
#fit_log=model.fit(X_train, y_train, batch_size=batch_size,
#                  epochs=n_epochs, validation_split=val_split,
#                  callbacks=[cl])
#if you want to use early stopping, uncomment the following lines
es=EarlyStopping(monitor='val_loss',patience=7,verbose=1)
fit_log=model.fit(X_train, y_train, batch_size=batch_size,
                  epochs=n_epochs, validation_split=val_split,
                  callbacks=[cl, es])

#### Plot of loss/accuracy during training  

In [ ]:
plt.plot(fit_log.history['loss'], label='train')
plt.plot(fit_log.history['val_loss'], label='valid')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(fit_log.history['accuracy'], label='train')
plt.plot(fit_log.history['val_accuracy'], label='valid')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#### Plot of loss/accuracy during training (from history file)  

In [ ]:
h=np.loadtxt(history_file, delimiter=',', skiprows=1)
epoch=h[:,0]
acc=h[:,1]
loss=h[:,2]
val_acc=h[:,3]
val_loss=h[:,4]

plt.plot(epoch, loss, label='train')
plt.plot(epoch, val_loss, label='valid')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epoch, acc, label='train')
plt.plot(epoch, val_acc, label='valid')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#### Accuracy against test data (Generalization performance)  

In [ ]:
score=model.evaluate(X_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

#### Save model file  

In [ ]:
model.save(model_file)